In [30]:
from PIL import Image
import os

# Load the TIFF file
for dirpath, dirnames, filenames in os.walk('/Users/kvakkefly/projects/joachim/open-elevation/data'):
    for filename in filenames:
        full_path = os.path.join(dirpath, filename)    
        if full_path.endswith('tif'):
            with Image.open(full_path) as img:
                if img.mode == 'F':
                    # Convert to 'RGB' for color images or 'L' for grayscale
                    img = img.convert('L')  # or img.convert('L') for grayscale

                    # Get image dimensions
                    width, height = img.size

                    # List to hold new pixel values
                    new_pixels = []

                    # Scale factor
                    scale_factor = 60  # Example scale factor

                    # Iterate over each pixel
                    for y in range(height):
                        for x in range(width):
                            value = img.getpixel( (x, y) )
                            
                            new_pixels.append(int(value / scale_factor))
                new_img = Image.new('L', img.size)
                new_img.putdata(new_pixels)
                # new_img.save('path/to/save/rescaled_rgb_image.png')
                new_file_name = f'heightdata/{filename.replace(".tif", ".png")}'
                new_img.save(new_file_name)
                print("Saved ", new_file_name)

Saved  heightdata/6701_50m_33.png
Saved  heightdata/7002_50m_33.png
Saved  heightdata/7806_50m_33.png
Saved  heightdata/6502_50m_33.png
Saved  heightdata/7404_50m_33.png
Saved  heightdata/7304_50m_33.png
Saved  heightdata/66M1_50m_33.png
Saved  heightdata/6900_50m_33.png
Saved  heightdata/7607_50m_33.png
Saved  heightdata/7906_50m_33.png
Saved  heightdata/7102_50m_33.png
Saved  heightdata/6601_50m_33.png
Saved  heightdata/7707_50m_33.png
Saved  heightdata/67M1_50m_33.png
Saved  heightdata/7204_50m_33.png
Saved  heightdata/7504_50m_33.png
Saved  heightdata/6800_50m_33.png
Saved  heightdata/7910_50m_33.png
Saved  heightdata/6500_50m_33.png
Saved  heightdata/7203_50m_33.png
Saved  heightdata/7608_50m_33.png
Saved  heightdata/7503_50m_33.png
Saved  heightdata/7000_50m_33.png
Saved  heightdata/6703_50m_33.png
Saved  heightdata/7711_50m_33.png
Saved  heightdata/7605_50m_33.png
Saved  heightdata/7809_50m_33.png
Saved  heightdata/6902_50m_33.png
Saved  heightdata/6603_50m_33.png
Saved  heightd

KeyboardInterrupt: 

In [40]:
from tifffile import imread, imsave
import numpy as np

def scale_image(image, clip_at):
    image = image/clip_at
    image = np.clip(image, 0, 1)
    image = (image * 255).astype(np.uint8)
    return image

for dirpath, dirnames, filenames in os.walk('/Users/kvakkefly/projects/joachim/open-elevation/data'):
    for filename in filenames:
        full_path = os.path.join(dirpath, filename)    
        if full_path.endswith('tif'):
            image = imread(full_path)
            image = scale_image(image, clip_at = 60)
            print("Reducing file size on ", full_path)

            new_img = Image.new('L', (image.shape[1], image.shape[0]))
            new_img.putdata(image.flatten().tolist())
            new_file_name = f'heightdata/{filename.replace(".tif", ".png")}'
            new_img.save(new_file_name)
            print("Saved ", new_file_name)
#            imsave('heightdata/'+filename, image, compression ='zlib')
            #imsave('heightdata/'+filename, image)

Reducing file size on  /Users/kvakkefly/projects/joachim/open-elevation/data/6701_50m_33.tif
Saved  heightdata/6701_50m_33.png
Reducing file size on  /Users/kvakkefly/projects/joachim/open-elevation/data/7002_50m_33.tif
Saved  heightdata/7002_50m_33.png
Reducing file size on  /Users/kvakkefly/projects/joachim/open-elevation/data/7806_50m_33.tif
Saved  heightdata/7806_50m_33.png
Reducing file size on  /Users/kvakkefly/projects/joachim/open-elevation/data/6502_50m_33.tif
Saved  heightdata/6502_50m_33.png
Reducing file size on  /Users/kvakkefly/projects/joachim/open-elevation/data/7404_50m_33.tif
Saved  heightdata/7404_50m_33.png
Reducing file size on  /Users/kvakkefly/projects/joachim/open-elevation/data/7304_50m_33.tif
Saved  heightdata/7304_50m_33.png
Reducing file size on  /Users/kvakkefly/projects/joachim/open-elevation/data/66M1_50m_33.tif
Saved  heightdata/66M1_50m_33.png
Reducing file size on  /Users/kvakkefly/projects/joachim/open-elevation/data/6900_50m_33.tif
Saved  heightdata/

AttributeError: 'tuple' object has no attribute 'transpose'

In [ ]:
from PIL import Image
files = []

# for dirpath, dirnames, filenames in os.walk('../open-elevation/data/'):
for dirpath, dirnames, filenames in os.walk('heightdata/'):
    for filename in filenames:
        full_path = os.path.join(dirpath, filename)    
        
        print(full_path)
        if full_path.endswith('tfw'):
            image_file_name = full_path.replace(".tfw", ".png")
            image = Image.open(image_file_name)
            with open(full_path, 'r') as f:
                lines = f.readlines()
                delta_x = float(lines[0])
                delta_y = float(lines[3])
                x0 = float(lines[4]) - delta_x * 0.5
                y0 = float(lines[5]) - delta_y * 0.5
                x1 = x0 + delta_x * image.size[1]
                y1 = y0 + delta_y * image.size[0]
                files.append({
                    'filename': image_file_name,
                    'pixel_size': (delta_x, delta_y),
                    'coords': ( y1,
                                y0,
                                x0,
                                x1,)
                })
                print(x0, y0, x1, y1)

In [ ]:
import math

def latlon_to_utm33(latitude, longitude):
    # Constants for WGS84 ellipsoid
    a = 6378137.0  # semi-major axis in meters
    f = 1 / 298.257223563  # flattening
    e_sq = 2 * f - f**2  # square of eccentricity
    k0 = 0.9996  # scale factor
    lon_origin = 15  # central meridian for UTM Zone 33
    
    # Convert latitude and longitude from degrees to radians
    lat_rad = math.radians(latitude)
    lon_rad = math.radians(longitude)
    
    N = a / math.sqrt(1 - e_sq * math.sin(lat_rad)**2)
    T = math.tan(lat_rad)**2
    C = e_sq / (1 - e_sq) * math.cos(lat_rad)**2
    A = math.cos(lat_rad) * (lon_rad - math.radians(lon_origin))
    
    M = a * ((1 - e_sq / 4 - 3 * e_sq**2 / 64 - 5 * e_sq**3 / 256) * lat_rad - 
             (3 * e_sq / 8 + 3 * e_sq**2 / 32 + 45 * e_sq**3 / 1024) * math.sin(2 * lat_rad) + 
             (15 * e_sq**2 / 256 + 45 * e_sq**3 / 1024) * math.sin(4 * lat_rad) - 
             (35 * e_sq**3 / 3072) * math.sin(6 * lat_rad))
    
    x = k0 * N * (A + (1 - T + C) * A**3 / 6 + 
                  (5 - 18 * T + T**2 + 72 * C - 58 * e_sq) * A**5 / 120)
    
    y = k0 * (M + N * math.tan(lat_rad) * (A**2 / 2 + (5 - T + 9 * C + 4 * C**2) * A**4 / 24 + 
                                           (61 - 58 * T + T**2 + 600 * C - 330 * e_sq) * A**6 / 720))
    
    # Adjust for northern and southern hemispheres
    if latitude < 0:
        y += 10000000  # 10 million meter offset for southern hemisphere
    
    x += 500000  # 500,000 meter offset for all UTM zones
    
    return x, y


In [ ]:
def find_file(x, y):
    for file in files:
        ( y0, y1, x0, x1) = file['coords']
        if x0 <= x <= x1 and y0 <= y <= y1:
            return file
    return None

def find_height(latitude, longitude):
    """
    Find the height of the terrain at the given longitude and latitude.
    :param longitude:
    :param latitude:
    :return:
    """
    x, y = latlon_to_utm33(latitude, longitude)
    print("Found utm33", x,y)
    file = find_file(x, y)
    image = Image.open(file['filename'])
    ( y0, y1, x0, x1) = file['coords']
    (delta_x, delta_y) = file['pixel_size']
    x_index = int((x - x0) / delta_x)
    y_index = int((y1 - y) / abs(delta_y))
    print(x_index, y_index, file['filename'])
    return image.getpixel( (y_index, x_index) )
    
find_height(59.004844, 11.063953)